In [1]:
from architectures import *
from dataset_numpy import *
from utils import *
from torch.utils.data import Subset
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
import json

In [2]:
#getting the configurations from the config file
with open('working_config.json', 'r') as f:
    config = json.load(f)

In [9]:
testset = RandomSeqDataset(r"D:\studium_tests\small_original_dataset2.npz",config["testset_key"],cuda=False)
trainset = RandomSeqDataset(r"D:\studium_tests\small_original_dataset2.npz",config["trainset_key"],cuda = False)


# Create dataloaders from each subset
test_loader = DataLoader(testset,  # we want to load our dataset
                         shuffle=False,  # shuffle for training
                         batch_size=1,  # 1 sample at a time
                         num_workers=0,
                         collate_fn=stack_if_possible_collate_fn  # no background workers
                         )

training_loader = DataLoader(trainset,  # we want to load our dataset
                             shuffle=False,  # shuffle for training
                             batch_size=4,  # stack 4 samples to a minibatch
                             num_workers=0,
                             collate_fn=stack_if_possible_collate_fn  # 2 background workers
                             )

In [24]:
#claring some folders
for key in config["to_clear"]:
    clear_folder(config["to_clear"][key])

In [11]:
class CNN(nn.Module):
    def __init__(self, n_input_channels: int, n_hidden_layers: int,
                 n_hidden_kernels: int, n_output_channels: int):
        """CNN, consisting of `n_hidden_layers` linear layers, using relu
        activation function in the hidden CNN layers.

        Parameters
        ----------
        n_input_channels: int
            Number of features channels in input tensor
        n_hidden_layers: int
            Number of hidden layers
        n_hidden_kernels: int
            Number of kernels in each hidden layer
        n_output_channels: int
            Number of features in output tensor
        """
        super(CNN, self).__init__()

        # encoder
        hidden_layers = []
        for _ in range(n_hidden_layers):
            # Add a CNN layer
            layer = nn.Conv2d(in_channels=n_input_channels,
                              out_channels=n_hidden_kernels,
                              kernel_size=7)
            hidden_layers.append(layer)
            # Add relu activation module to list of modules
            hidden_layers.append(nn.ReLU())
            n_input_channels = n_hidden_kernels

        # decoder
        for _ in range(n_hidden_layers + 1):
            # Add a CNN layer
            layer = nn.ConvTranspose2d(in_channels=n_input_channels,
                                       out_channels=n_hidden_kernels,
                                       kernel_size=7)
            hidden_layers.append(layer)
            # Add relu activation module to list of modules
            hidden_layers.append(nn.ReLU())
            n_input_channels = n_hidden_kernels

        self.hidden_layers = nn.Sequential(*hidden_layers)

        self.output_layer = nn.Conv2d(in_channels=n_input_channels,
                                      out_channels=n_output_channels,
                                      kernel_size=7)
        self.linear1 = nn.Linear(90*90, 90*90)

    def forward(self, x):
        """Apply CNN to `x`

        Parameters
        ----------
        x: torch.tensor
            Input tensor of shape (n_samples, n_input_channels, x, y)

        Returns
        ----------
        torch.tensor
            Output tensor of shape (n_samples, n_output_channels, u, v)
        """
        # Apply hidden layers module
        hidden_features = self.hidden_layers(x)

        # Apply last layer (=output layer)
        output = self.output_layer(hidden_features)
        output = x.view(-1,1,90*90)
        output = self.linear1(output)
        output = torch.sigmoid(output)
        output = output.view(-1,1,90,90)
        return output

net = CNN(n_input_channels=config["network_config"]["n_input_channels"], n_hidden_layers=config["network_config"]["n_hidden_layers"], n_hidden_kernels=config["network_config"]["n_hidden_kernels"],
          n_output_channels=config["network_config"]["n_output_channels"])
torch.save(net, os.path.join(config["results_path"], "my_pc_cnn.pt"))

In [25]:
# Create an instance of our CNN
cnn = torch.load(os.path.join(config["results_path"], "my_pc_cnn.pt"))

# GPU will be much faster here
device = torch.device(config["device"])
#device = torch.device("cpu")
cnn.to(device=device)

# also try rmse and absolute error
loss_function = torch.nn.MSELoss(reduction="mean")

# Use a SGD optimizer
#optimizer = torch.optim.SGD(cnn.parameters(), lr=1e-4)
optimizer = torch.optim.Adam(cnn.parameters(), lr=1e-4)

# Define a tensorboard summary writer that writes to directory "results_path/tensorboard"
writer = SummaryWriter(log_dir=os.path.join(config["results_path"], 'tensorboard'), flush_secs=1)

best_loss = np.float("inf")
# Optimize our dsnn model using SGD:
print("Continuing the training:")

#print(f"cnn: {cnn.device}")
# for update in tqdm(range(200)):
with tqdm(total=config["n_updates"]) as pbar:
    for update in range(config["n_updates"]):
        x = 0
        for data in training_loader:
            # Compute the output
            input_array,  original = data

            # ic(input_array.unsqueeze(1).type(torch.float32).shape)
            output = cnn(input_array.unsqueeze(1).type(torch.float32))

            # Compute the loss
            loss = loss_function(output, original.type(torch.float32).unsqueeze(1))
            # Compute the gradients
            loss.backward()
            # Preform the update
            optimizer.step()
            # Reset the accumulated gradients
            optimizer.zero_grad()
            x += 1
            if x == 500:
                break

        # automatic saving of the best model
        if update % 1 == 0:
            with torch.no_grad():
                y = 0
                sum_loss = 0
                for tdata in test_loader:
                    input_array, original = data

                    output = cnn(input_array.unsqueeze(1).type(torch.float32))
                    sum_loss += loss_function(output, original.type(torch.float32).unsqueeze(1))
                    y += 1
                    if y == 50:
                        print(sum_loss)
                        break
                    
            if sum_loss < best_loss:
                torch.save(cnn, os.path.join(config["results_path"], "autosave", f"loss_{sum_loss}.pt"))
                best_loss = loss

        # Tensorboard
        if update % 1 == 0:
            # Add losse as scalars to tensorboard
            writer.add_scalar(tag="training/loss", scalar_value=sum_loss,
                              global_step=update)

            # Add images to Tensorboard
            writer.add_image(tag="training/output", img_tensor=output[0], global_step=update)

            writer.add_image(tag="training/input", img_tensor=(original.unsqueeze(1)[0]), global_step=update)

        torch.save(cnn, os.path.join(config["results_path"], "my_pc_cnn.pt"))
        pbar.update(1)

print("training finished")

  0%|          | 0/100 [00:00<?, ?it/s]

Continuing the training:
tensor(0.9897)


  1%|          | 1/100 [06:55<11:25:15, 415.30s/it]

tensor(0.9549)


  2%|▏         | 2/100 [14:04<11:31:58, 423.66s/it]

...

tensor(0.8921)


  5%|▌         | 5/100 [35:17<11:09:04, 422.57s/it]

tensor(0.8769)


  6%|▌         | 6/100 [48:45<12:43:59, 487.65s/it]


KeyboardInterrupt: 

In [21]:
1e-4


0.0001